In [2]:
# import libraries

import numpy as np
import pandas as pd
import os
from os import listdir
from os.path import isfile, join

import datetime as dt
import time

import yfinance as yf

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import cufflinks as cf
import plotly.express as px
import plotly.graph_objects as go
# make plotly work in jupyter notebook
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
cf.go_offline()
from plotly.subplots import make_subplots

from datetime import date
from datetime import timedelta

import warnings
warnings.simplefilter('ignore')

In [1]:
# stock data files downloaded from yfinance:
path = 'C://Users//sophi//Documents//Portfolio_Projects//DS_for_Investing//TSE Stock Data//'

# open each stock's csv file and add returns, cum returns, bolinger band and ichimoku data
# (to be used for plotting the data later)

In [3]:
def get_data_from_csv(ticker):
    '''
    Read csv and save to a new dataframe
    '''
    try:
        df = pd.read_csv(path + ticker + '.csv', index_col=0)
    except Exception as e:
        print(e)
    
    return df

In [4]:
def add_daily_return_to_df(df):
    '''
    Adds a daily return column to the df
    '''
    df['daily_return'] = (df['Close'] / df['Close'].shift(1)) - 1
    return df


def add_cum_return_to_df(df):
    '''
    Adds cumulative return column to df
    '''
    df['cum_return'] = (1 + df['daily_return']).cumprod()
    return df

In [5]:
def add_bolinger_bands(df):
    '''
    Adds a bolinger band column to the dataframe
    '''
    
    df['middle_band'] = df['Close'].rolling(window=20).mean()
    df['upper_band'] = df['middle_band'] + 1.96 * df['Close'].rolling(window=20).std()
    df['lower_band'] = df['middle_band'] - 1.96 * df['Close'].rolling(window=20).std()
    return df

### Add Ichimoku Data to Dataframe

The Ichimoku (One Look) is considered an all in one indicator. It provides information on momentum, support and resistance. It is made up of 5 lines. Long term traders focus on day or weekly data.

 - Conversion Line (Tenkan-sen) : Represents support, resistance and reversals. Used to measure short term trends.
 - Baseline (Kijun-sen) : Represents support, resistance and confirms trend changes. Allows you to evaluate the strength of medium term trends. Called the baseline because it lags the price.
 - Leading Span A (Senkou A) : Used to identify future areas of support and resistance
 - Leading Span B (Senkou B) : Other line used to identify suture support and resistance
 - Lagging Span (Chikou) : Shows possible support and resistance. It is used to confirm signals obtained from other lines.
 - Cloud (Kumo) : Space between Span A and B. Represents the divergence in price evolution.

In [21]:
def add_Ichimoku(df):
    '''
    Add Ichimoku Data to Dataframe
    '''
    # Lagging Span = Price shifted back 26 periods
    df['Lagging'] = df['Close'].shift(-26)
    
    # Base Line = (Highest Value in period + Lowest value in period)/2 (26 Sessions)
    hi_val2 = df['High'].rolling(window=26).max()
#     low_val2 = df['Low'].rolling(window=26).min()
#     df['Baseline'] = (hi_val2 + low_val2) / 2
    
    # Conversion Line = (Highest Value in period + Lowest value in period)/2 (9 Sessions)
#     hi_val = df['High'].rolling(window=9).max()
#     low_val = df['Low'].rolling(window=9).min()
#     df['Conversion'] = (hi_val + low_val) / 2
    
#     # Leading Span A = (Conversion Value + Base Value)/2 (26 Sessions)
#     df['SpanA'] = ((df['Conversion'] + df['Baseline']) / 2)
    
#     # Leading Span B = (Conversion Value + Base Value)/2 (52 Sessions)
#     hi_val3 = df['High'].rolling(window=52).max()
#     low_val3 = df['Low'].rolling(window=52).min()
#     df['SpanB'] = ((hi_val3 + low_val3) / 2).shift(26)

    return df

##  Set Variables

In [10]:
# folder to find and resave dataframes
path = 'C://Users//sophi//Documents//Portfolio_Projects//DS_for_Investing//TSE Stock Data//'

# get list of tickers in folder
files = [x for x in listdir(path) if isfile(join(path, x))]
tickers = [os.path.splitext(x)[0] for x in files]
print(len(tickers))
tickers[:5]

1023


['AAG_V', 'AAU_V', 'AAV_TO', 'ABRA_V', 'ABR_V']

## Test

In [22]:
# test on one file before running for all 1000 TSE stocks

try: 
    ticker = 'AAG_V'
    print('Working on file', ticker)
    new_df = get_data_from_csv(ticker)
    new_df = add_daily_return_to_df(new_df)
    new_df = add_cum_return_to_df(new_df)
    new_df = add_bolinger_bands(new_df)
    new_df = add_Ichimoku(new_df)
#     new_df.to_csv(path + ticker + '.csv')
    print('Success: File', ticker, 'ammended and saved')
    display(new_df.head())
except Exception as e:
    print(e)

Working on file AAG_V
Success: File AAG_V ammended and saved


,Open,High,Low,Close,Volume,Dividends,Stock Splits,daily_return,cum_return,middle_band,upper_band,lower_band,Lagging
Date,,,,,,,,,,,,,
2017-09-25,0.04,0.04,0.04,0.04,0,0,0,NaN,NaN,NaN,NaN,NaN,0.140
2017-09-26,0.04,0.04,0.04,0.04,0,0,0,0.0,1.0,NaN,NaN,NaN,0.150
2017-09-27,0.04,0.04,0.04,0.04,0,0,0,0.0,1.0,NaN,NaN,NaN,0.140
2017-09-28,0.04,0.04,0.04,0.04,0,0,0,0.0,1.0,NaN,NaN,NaN,0.190
2017-09-29,0.04,0.04,0.04,0.04,0,0,0,0.0,1.0,NaN,NaN,NaN,0.175


In [23]:
# now run for all 1023 tickers in TSE folder

for ticker in tickers:
    try: 
        print('Working on file', ticker)
        new_df = get_data_from_csv(ticker)
        new_df = add_daily_return_to_df(new_df)
        new_df = add_cum_return_to_df(new_df)
        new_df = add_bolinger_bands(new_df)
        new_df = add_Ichimoku(new_df)
        new_df.to_csv(path + ticker + '.csv')
        print(f'Success: File {ticker} ammended and saved')
#         display(new_df.head())
    except Exception as e:
        print(e)

Working on file AAG_V
Success: File AAG_V ammended and saved
Working on file AAU_V
Success: File AAU_V ammended and saved
Working on file AAV_TO
Success: File AAV_TO ammended and saved
Working on file ABRA_V
Success: File ABRA_V ammended and saved
Working on file ABR_V
Success: File ABR_V ammended and saved
Working on file ABST_TO
Success: File ABST_TO ammended and saved
Working on file ABXX_NE
Success: File ABXX_NE ammended and saved
Working on file ABX_TO
Success: File ABX_TO ammended and saved
Working on file ACB_TO
Success: File ACB_TO ammended and saved
Working on file ACDCCSE
Success: File ACDCCSE ammended and saved
Working on file ACO-X_TO
Success: File ACO-X_TO ammended and saved
Working on file ACOG_V
Success: File ACOG_V ammended and saved
Working on file ACQ_TO
Success: File ACQ_TO ammended and saved
Working on file ACST_V
Success: File ACST_V ammended and saved
Working on file ACU_V
Success: File ACU_V ammended and saved
Working on file AC_TO
Success: File AC_TO ammended an

Success: File BRG_V ammended and saved
Working on file BRMI_TO
Success: File BRMI_TO ammended and saved
Working on file BRND-A-U_NE
Success: File BRND-A-U_NE ammended and saved
Working on file BRO_V
Success: File BRO_V ammended and saved
Working on file BSR_V
Success: File BSR_V ammended and saved
Working on file BSX_TO
Success: File BSX_TO ammended and saved
Working on file BTB-UN_TO
Success: File BTB-UN_TO ammended and saved
Working on file BTCG-UN_TO
Success: File BTCG-UN_TO ammended and saved
Working on file BTE_TO
Success: File BTE_TO ammended and saved
Working on file BTO_TO
Success: File BTO_TO ammended and saved
Working on file BTR_V
Success: File BTR_V ammended and saved
Working on file BU_TO
Success: File BU_TO ammended and saved
Working on file BYD_TO
Success: File BYD_TO ammended and saved
Working on file BYL_TO
Success: File BYL_TO ammended and saved
Working on file BZ_V
Success: File BZ_V ammended and saved
Working on file CAD_V
Success: File CAD_V ammended and saved
Work

Success: File DOL_TO ammended and saved
Working on file DOO_TO
Success: File DOO_TO ammended and saved
Working on file DPM_TO
Success: File DPM_TO ammended and saved
Working on file DRDR_TO
Success: File DRDR_TO ammended and saved
Working on file DRM_TO
Success: File DRM_TO ammended and saved
Working on file DRT_TO
Success: File DRT_TO ammended and saved
Working on file DR_TO
Success: File DR_TO ammended and saved
Working on file DSG_TO
Success: File DSG_TO ammended and saved
Working on file DSLV_V
Success: File DSLV_V ammended and saved
Working on file DSV_V
Success: File DSV_V ammended and saved
Working on file DTG_V
Success: File DTG_V ammended and saved
Working on file DV_V
Success: File DV_V ammended and saved
Working on file DXT_TO
Success: File DXT_TO ammended and saved
Working on file DYA_TO
Success: File DYA_TO ammended and saved
Working on file EAM_V
Success: File EAM_V ammended and saved
Working on file ECN_TO
Success: File ECN_TO ammended and saved
Working on file ECOM_V
Su

Success: File GLDL_V ammended and saved
Working on file GLDX_V
Success: File GLDX_V ammended and saved
Working on file GLD_V
Success: File GLD_V ammended and saved
Working on file GLO_TO
Success: File GLO_TO ammended and saved
Working on file GLXY_TO
Success: File GLXY_TO ammended and saved
Working on file GMA_V
Success: File GMA_V ammended and saved
Working on file GMIN_V
Success: File GMIN_V ammended and saved
Working on file GMTN_V
Success: File GMTN_V ammended and saved
Working on file GMX_TO
Success: File GMX_TO ammended and saved
Working on file GOG_V
Success: File GOG_V ammended and saved
Working on file GOLD_TO
Success: File GOLD_TO ammended and saved
Working on file GOOS_TO
Success: File GOOS_TO ammended and saved
Working on file GPH_V
Success: File GPH_V ammended and saved
Working on file GPR_TO
Success: File GPR_TO ammended and saved
Working on file GPV_V
Success: File GPV_V ammended and saved
Working on file GRAM-U_NE
Success: File GRAM-U_NE ammended and saved
Working on fi

Success: File LBS_TO ammended and saved
Working on file LB_TO
Success: File LB_TO ammended and saved
Working on file LEAF_TO
Success: File LEAF_TO ammended and saved
Working on file LGD_TO
Success: File LGD_TO ammended and saved
Working on file LGO_TO
Success: File LGO_TO ammended and saved
Working on file LIF_TO
Success: File LIF_TO ammended and saved
Working on file LIO_V
Success: File LIO_V ammended and saved
Working on file LIS_V
Success: File LIS_V ammended and saved
Working on file LITH_V
Success: File LITH_V ammended and saved
Working on file LI_V
Success: File LI_V ammended and saved
Working on file LLG_V
Success: File LLG_V ammended and saved
Working on file LNR_TO
Success: File LNR_TO ammended and saved
Working on file LN_TO
Success: File LN_TO ammended and saved
Working on file LORD_V
Success: File LORD_V ammended and saved
Working on file LPEN_TO
Success: File LPEN_TO ammended and saved
Working on file LPS_V
Success: File LPS_V ammended and saved
Working on file LR_V
Succes

Success: File NWC_TO ammended and saved
Working on file NWH-UN_TO
Success: File NWH-UN_TO ammended and saved
Working on file NWST_V
Success: File NWST_V ammended and saved
Working on file NWX_V
Success: File NWX_V ammended and saved
Working on file NXE_TO
Success: File NXE_TO ammended and saved
Working on file NXO_V
Success: File NXO_V ammended and saved
Working on file NXR-UN_TO
Success: File NXR-UN_TO ammended and saved
Working on file N_V
Success: File N_V ammended and saved
Working on file OBE_TO
Success: File OBE_TO ammended and saved
Working on file OCG_V
Success: File OCG_V ammended and saved
Working on file OCO_V
Success: File OCO_V ammended and saved
Working on file OGC_TO
Success: File OGC_TO ammended and saved
Working on file OGI_TO
Success: File OGI_TO ammended and saved
Working on file OGN_V
Success: File OGN_V ammended and saved
Working on file OGO_V
Success: File OGO_V ammended and saved
Working on file OG_V
Success: File OG_V ammended and saved
Working on file OLA_TO
Su

Success: File ROMJ_V ammended and saved
Working on file ROOT_TO
Success: File ROOT_TO ammended and saved
Working on file ROS_V
Success: File ROS_V ammended and saved
Working on file ROXG_TO
Success: File ROXG_TO ammended and saved
Working on file RPX_V
Success: File RPX_V ammended and saved
Working on file RRI_V
Success: File RRI_V ammended and saved
Working on file RSI_TO
Success: File RSI_TO ammended and saved
Working on file RSLV_V
Success: File RSLV_V ammended and saved
Working on file RSS_V
Success: File RSS_V ammended and saved
Working on file RUS_TO
Success: File RUS_TO ammended and saved
Working on file RVG_V
Success: File RVG_V ammended and saved
Working on file RVX_TO
Success: File RVX_TO ammended and saved
Working on file RWR_V
Success: File RWR_V ammended and saved
Working on file RW_V
Success: File RW_V ammended and saved
Working on file RY_TO
Success: File RY_TO ammended and saved
Working on file SAP_TO
Success: File SAP_TO ammended and saved
Working on file SBB_TO
Succes

Success: File TRZ_TO ammended and saved
Working on file TSK_TO
Success: File TSK_TO ammended and saved
Working on file TSL_TO
Success: File TSL_TO ammended and saved
Working on file TSN_V
Success: File TSN_V ammended and saved
Working on file TSU_TO
Success: File TSU_TO ammended and saved
Working on file TTR_V
Success: File TTR_V ammended and saved
Working on file TUD_V
Success: File TUD_V ammended and saved
Working on file TUO_V
Success: File TUO_V ammended and saved
Working on file TVE_TO
Success: File TVE_TO ammended and saved
Working on file TWM_TO
Success: File TWM_TO ammended and saved
Working on file TXG_TO
Success: File TXG_TO ammended and saved
Working on file TXP_TO
Success: File TXP_TO ammended and saved
Working on file UCU_V
Success: File UCU_V ammended and saved
Working on file UFS_TO
Success: File UFS_TO ammended and saved
Working on file UGD_V
Success: File UGD_V ammended and saved
Working on file UGE_V
Success: File UGE_V ammended and saved
Working on file UNI_TO
Succes

In [24]:
print(ticker) # looking at final ticker in list 
new_df.head()

ZZZ_TO


,Open,High,Low,Close,Volume,Dividends,Stock Splits,daily_return,cum_return,middle_band,upper_band,lower_band,Lagging
Date,,,,,,,,,,,,,
2017-09-25,31.311620,31.556925,30.637026,30.838528,92900,0.0,0,NaN,NaN,NaN,NaN,NaN,33.817253
2017-09-26,30.838531,31.250294,30.838531,31.206490,80200,0.0,0,0.011932,1.011932,NaN,NaN,NaN,29.419264
2017-09-27,31.224011,31.670821,31.215252,31.355427,65000,0.0,0,0.004773,1.016761,NaN,NaN,NaN,29.629517
2017-09-28,31.355427,31.854800,31.232774,31.495602,90800,0.0,0,0.004471,1.021307,NaN,NaN,NaN,29.025019
2017-09-29,31.548165,31.828519,31.031270,31.609495,64300,0.0,0,0.003616,1.025000,NaN,NaN,NaN,28.385468
